In [91]:
from forward_backward_AD import *

import numpy as np
import torch
import time, timeit

import scipy.io

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
workspace = scipy.io.loadmat(r'../Data Sets/Debug Data/validation_python_workspace.mat')

c:\Users\Bruker\anaconda3\envs\SAXSTT\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [93]:
print(workspace.keys())

dict_keys(['__header__', '__version__', '__globals__', 'opt_inputs', 'p', 's', 'projection', 'E', 'numOfCoeffs', 'skip_optimization', 'None', 'return_synth_proj', 'return_Ereg', 'find_coefficients', 'find_orientation', 'find_grad', 'phi_det', 'theta_det', 'l', 'm', 'nx', 'ny', 'nz', 'numOfsegments', 'numOfvoxels', 'mask3D', 'numOfpixels', 'numOfordersopt', 'theta_struct', 'phi_struct', 'a', 'ii', 'grad_a', 'grad_theta_struct', 'grad_phi_struct', 'zeros_struct', 'Ylm_coef', 'a_temp', 'a_temp1', 'a_temp2', 'ones_struct', 'unit_q_beamline', 'N', 'x', 'y', 'z', 'X', 'Y', 'Z', 'current_projection', '__function_workspace__'])


In [94]:
theta_struct_it = workspace['theta_struct']
phi_struct_it = workspace['phi_struct']
a_temp_it =    workspace['a_temp']
ny = workspace['ny']
nx = workspace['nx']
nz = workspace['nz']
numOfsegments = workspace['numOfsegments']
current_projection = workspace['projection'][0,0]#[2]
p = workspace['p']
X = workspace['X']
Y = workspace['Y']
Z = workspace['Z']
numOfpixels = workspace['numOfpixels']
unit_q_beamline = workspace['unit_q_beamline']
Ylm_coef = workspace['Ylm_coef']
find_coefficients = workspace['find_coefficients']
find_orientation = workspace['find_orientation']
numOfCoeffs = workspace['numOfCoeffs']
numOfvoxels = workspace['numOfvoxels']



In [101]:
print(p['GPU'])

[[array([[0]], dtype=uint8)]]


In [5]:
#print(current_projection[0,0][2].shape)
# Testing shit

print(type(numOfvoxels))

yo = int(np.squeeze(numOfvoxels))

print(yo)

print(type(yo))

new_tens = torch.tensor([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], dtype=torch.float64).reshape(4,4)
print(new_tens)


A = torch.rand((3,3,64))
B = torch.rand((3,8))
#C = torch.matmul(A[0], B) # Have to figure out how this works. 

projection_data = torch.tensor(
        current_projection["data"])

print(projection_data.shape)

print(current_projection["data"].shape)
print(current_projection["Rot_exp"].shape)


<class 'numpy.ndarray'>
64
<class 'int'>
tensor([[ 1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.],
        [ 9., 10., 11., 12.],
        [13., 14., 15., 16.]], dtype=torch.float64)
torch.Size([4, 4, 8])
(4, 4, 8)
(3, 3)


Done debugging. Error is at least correct. 

In [12]:
%timeit main(theta_struct_it, phi_struct_it, a_temp_it, ny, nx, nz, numOfsegments, current_projection, p, X, Y, Z, numOfpixels, unit_q_beamline, Ylm_coef, find_coefficients, find_orientation, numOfCoeffs, numOfvoxels, find_grad=True)

48.3 ms ± 2.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [146]:
x = np.arange(64).reshape(4,4,4)
y = np.arange(64).reshape(4,4,4)

lower = np.argwhere( x>10)
upper = np.argwhere(x<20)

y = np.argwhere( (x>10) & (x<20) & (y>15) & (y<25) )
print(y)

# Next is out of bounds. Index 5 found when size is 4, index 3 is therefore max. 
print(x[y[:,0], y[:,1], y[:,2] ])

[[1 0 0]
 [1 0 1]
 [1 0 2]
 [1 0 3]]
[16 17 18 19]


In [93]:

# print(X)
# print('\n')
# print(Y)
proj = torch.zeros((7,7,8))
Ax =  np.floor(
    X - (-1.38545603) + 1
    ).astype(int)
 # Drop +1 due to 0-indexing. AI suggested, but believe I should copy cpp-code
Ay = np.floor(Y - (-0.19120833) + 1).astype(int)

# print(Ax)
# print(Ay)

indices = np.argwhere( (Ax>0) & (Ax<7) & (Ay>0) & (Ay<7) )

# print(Ax[indices[:,0], indices[:,1], indices[:,2] ])
# print(Ay[indices[:,0], indices[:,1], indices[:,2] ])
#print(indices[:,:])
np.random.seed(0)
temp = np.random.random(len(indices[:,0])  )

temp = torch.from_numpy(temp).unsqueeze(1)
temp = temp.expand(-1,8)
tomo = torch.ones((7,7,7,8))

proj[Ay[indices[:,0], indices[:,1], indices[:,2] ], Ax[indices[:,0], indices[:,1], indices[:,2] ] ] +=  tomo[indices[:,0], indices[:,1], indices[:,2]] * temp

print( (temp* tomo[indices[:,0], indices[:,1], indices[:,2]]).shape ) # Why is the shape 140 times 140??? Some weird broadcasting going on.
print(proj[Ay[indices[:,0], indices[:,1], indices[:,2] ], Ax[indices[:,0], indices[:,1], indices[:,2] ] ].shape)
print(np.transpose(proj, (2,0,1)).shape )

torch.Size([140, 8])
torch.Size([140, 8])
torch.Size([8, 7, 7])


In [94]:
print( (Ay[indices[:,0], indices[:,1], indices[:,2] ], Ax[indices[:,0], indices[:,1], indices[:,2] ]))
print( (Ay[indices[:,0], indices[:,1], indices[:,2] ] -1, Ax[indices[:,0], indices[:,1], indices[:,2] ]) )
print( (Ay[indices[:,0], indices[:,1], indices[:,2] ], Ax[indices[:,0], indices[:,1], indices[:,2] ] -1) )
print( (Ay[indices[:,0], indices[:,1], indices[:,2] ] -1, Ax[indices[:,0], indices[:,1], indices[:,2] ] -1) )

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4]), array([1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4,
       4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5,
       5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4,
       4, 5, 5, 5, 5, 5, 5, 5]))
(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [201]:
# Check out nan values

permuted_data = np.load(r"../permuted_data.npy") #np.load(r'../Data Sets/Debug Data/permuted_data.npy')
proj_out_all = np.load(r"../proj_out_all.npy") #np.load(r'../Data Sets/Debug Data/proj_out_all.npy')
aux_diff_poisson = np.load(r"../aux_diff_poisson.npy") #np.load(r'../Data Sets/Debug Data/aux_diff_poisson.npy')

In [202]:
print(permuted_data.shape)
print(proj_out_all.shape)
print(aux_diff_poisson.shape)

(8, 4, 4)
(8, 4, 4)
(8, 4, 4)


In [209]:
print(permuted_data[:,0,0])
print(proj_out_all[:,0,0])
print(aux_diff_poisson[:,0,0])
# negative projection data...

[  0.69836503   0.839173    21.605843   216.13242    102.646805
   0.92443      0.6229484    0.5012255 ]
[-9.80411e-11 -9.80411e-11 -9.80411e-11 -9.80411e-11 -9.80411e-11
 -9.80411e-11 -9.80411e-11 -9.80411e-11]
[nan nan nan nan nan nan nan nan]


In [124]:
endgame = torch.zeros((2,2,8), dtype=torch.float64)
indices = torch.tensor( [[0,0], [0,0]] )
values = torch.ones( (2,8), dtype=torch.float64) 
#endgame[indices[:,0], indices[:,1], indices[:,2]] += values

#endgame[indices[:,0], indices[:,1], indices[:,2]] += values
#endgame.scatter_add_(0, indices, values)
endgame = torch.index_put(endgame, (indices[:,0], indices[:,1]), values, accumulate=True) #endgame.index_add(0, indices, values, alpha=1)


print(endgame) # This is the issue

tensor([[[2., 2., 2., 2., 2., 2., 2., 2.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=torch.float64)


In [14]:
projections = scipy.io.loadmat(r'../Data Sets/Debug Data/projections_all.mat')

In [15]:
print(projections.keys())

dict_keys(['__header__', '__version__', '__globals__', 'projection'])


In [23]:
projection = projections["projection"]
print(projection[:,:].shape)

(1, 255)


In [72]:
all_projection_data = projection["data"][0,:]

In [73]:
print(all_projection_data[0].shape)

(7, 7, 8)


In [71]:
indices = np.arange(0,2, dtype=int)
print(all_projection_data[indices].shape)
arr = np.zeros((2,7,7,8))
arr[indices, :, :, :] = all_projection_data[indices][:,:,:]


(2,)


IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [90]:
a = np.ones((2,2,2))
b = np.ones((2,2))

c = a*b
b[:,0] = 0
print(b)
print(np.ndarray.max(b, axis = 1))

[[0. 1.]
 [0. 1.]]
[1. 1.]


In [102]:
new_model_workspace = scipy.io.loadmat(r"../Data Sets/Debug Data/new_model_python_workspace.mat")
print(new_model_workspace.keys())

dict_keys(['__header__', '__version__', '__globals__', 'opt_inputs', 'p', 's', 'projection', 'E', 'skip_optimization', 'find_grad', 'None', 'phi_det', 'theta_det', 'nx', 'ny', 'nz', 'numOfsegments', 'numOfvoxels', 'mask3D', 'numOfpixels', 'theta_struct', 'phi_struct', 'A', 'B', 'unit_q_beamline', 'N', 'x', 'y', 'z', 'X', 'Y', 'Z', '__function_workspace__'])


c:\Users\Bruker\anaconda3\envs\SAXSTT\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [103]:
p_projection_filename = new_model_workspace["p"]["projection_filename"][0,0][0]

In [104]:
print(p_projection_filename)

C:\Users\Bruker\OneDrive\Dokumenter\NTNU\XRD_CT/Data sets/Validation_periodic_filter1_3cube_4off_0align.mat
